# Fine-tuning Llama-2 Model with HPU

In this Jupyter notebook, we will fine-tune a [Llama-2-7b](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) model by using HPU in DDP accelerate mode. We will use PyTorch for model training and Ray for distributed training. We will use dataset [tatsu-lab/alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca).

[Habana Gaudi AI Processors (HPUs)](https://habana.ai) are AI hardware accelerators designed by Habana Labs. For more information, see [Gaudi Architecture](https://docs.habana.ai/en/latest/Gaudi_Overview/index.html) and [Gaudi Developer Docs](https://developer.habana.ai/).

Basic features for this fine-tuning example are:
- Running on HPUs, support three execution mode: "lazy", "eager", "eager.compile".
- LoRA training.
- [`GaudiTrainer`](https://github.com/huggingface/optimum-habana/blob/main/optimum/habana/transformers/trainer.py) based training.
- Llama-2-7b model.
- Ray based scheduling and management.

## Prepare environment
A node with Gaudi/Gaudi2 installed is required to run this example. This example will use 4 workers to train the model, each using 1 HPU.

We recommend using a prebuilt container to run these examples. To run a container, you need Docker. See [Install Docker Engine](https://docs.docker.com/engine/install/) for installation instructions.

Next, follow [Run Using Containers](https://docs.habana.ai/en/latest/Installation_Guide/Bare_Metal_Fresh_OS.html?highlight=installer#run-using-containers) to install the Habana drivers and container runtime.

### Get docker image
``` bash
docker pull vault.habana.ai/gaudi-docker/1.15.1/ubuntu22.04/habanalabs/pytorch-installer-2.2.0:latest
```
### Run docker image
``` bash
docker run -it --runtime=habana -e HABANA_VISIBLE_DEVICES=all -e OMPI_MCA_btl_vader_single_copy_mechanism=none --cap-add=sys_nice --net=host --ipc=host vault.habana.ai/gaudi-docker/1.15.1/ubuntu22.04/habanalabs/pytorch-installer-2.2.0:latest
# maybe should mapping your workspace volumns
```
### Install dependency
``` bash
pip install ray[train] notebook transformers datasets evaluate peft accelerate optimum-habana
```

## Import necessary libraries

In [1]:
import os
import copy
import time
from typing import Dict

import torch
from torch import nn
from torch.utils.data import DataLoader

import datasets
import transformers
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    AutoModelForSequenceClassification,
)

from tqdm import tqdm

import peft

import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer
from ray.train.torch import TorchConfig

import habana_frameworks.torch.core as htcore

from optimum.habana import GaudiTrainer, GaudiConfig, GaudiTrainingArguments
from optimum.habana.transformers.modeling_utils import adapt_transformers_to_gaudi

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Prepare Dataset Function

Preprocessing the raw dataset's each line with specified format.

In [2]:

def preprocess_dataset(raw_datasets):

    PROMPT_DICT = {
        "prompt_with_input": (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
        ),
        "prompt_without_input": (
            "Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:"
        ),
    }

    def create_prompts(examples):
        prompts = {}
        prompts["source"] = []
        prompts["target"] = []
        for example in examples:
            prompt_template = (
                PROMPT_DICT["prompt_with_input"] if example["input"] != "" else PROMPT_DICT["prompt_without_input"]
            )
            source = prompt_template.format_map(example)
            prompts["source"].append(source)
            prompts["target"].append(example["output"])
        return prompts

    # Preprocessing the datasets.
    for key in raw_datasets:
        prompts = create_prompts(raw_datasets[key])
        columns_to_be_removed = list(raw_datasets[key].features.keys())
        raw_datasets[key] = raw_datasets[key].add_column("prompt_sources", prompts["source"])
        raw_datasets[key] = raw_datasets[key].add_column("prompt_targets", prompts["target"])
        raw_datasets[key] = raw_datasets[key].remove_columns(columns_to_be_removed)

## Dataset to Tokenizer Function

Tokenize each line in dataset by model tokenizer.

In example codes, we concatenate the dataset's line content to accelerate training speed.

All datasets are processed as "train" datasets, no evaluation datasets are sampled from raw_datasets.

In [3]:

def preprocess_dataset_to_tokenizer(raw_datasets, tokenizer):
    max_seq_length = 512
    tokenizer.pad_token_id = 0
    tokenizer.eos_token_id = 1
    tokenizer.bos_token_id = 2

    def tokenize(prompt, add_eos_token=True):
        results = tokenizer(
            prompt,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        for i in range(len(results["input_ids"])):
            if (
                results["input_ids"][i][-1] != tokenizer.eos_token_id
                and len(results["input_ids"][i]) < max_seq_length
                and add_eos_token
            ):
                results["input_ids"][i].append(tokenizer.eos_token_id)
                results["attention_mask"][i].append(1)

        results["labels"] = copy.deepcopy(results["input_ids"])
        results["input_id_len"] = [len(result) for result in results["input_ids"]]
        return results

    def preprocess_function(examples):
        keys = list(examples.data.keys())
        if len(keys) != 2:
            raise ValueError("Unsupported dataset format")

        st = [s + t for s, t in zip(examples[keys[0]], examples[keys[1]])]

        examples_tokenized = tokenize(st)
        input_ids = examples_tokenized["input_ids"]
        labels = examples_tokenized["labels"]
        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": examples_tokenized["attention_mask"],
        }

    tokenized_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        load_from_cache_file=True,
    )

    def concatenate_data(dataset, max_seq_length):
        concatenated_dataset = {}
        for column in dataset.features:
            concatenated_data = [item for sample in dataset[column] for item in sample]
            reshaped_data = [
                concatenated_data[i * max_seq_length : (i + 1) * max_seq_length]
                for i in range(len(concatenated_data) // max_seq_length)
            ]
            concatenated_dataset[column] = reshaped_data
        return datasets.Dataset.from_dict(concatenated_dataset)

    tokenized_datasets_ = tokenized_datasets["train"].remove_columns(["prompt_sources", "prompt_targets"])
    tokenized_datasets["train"] = concatenate_data(tokenized_datasets_, max_seq_length)

    return tokenized_datasets

## Training Function

This function will be executed by each worker during training, with following steps:

- loading datasets and preprocess datasets, just load the first 4096 item as training datasets.
- loading pretrained model as tokenizer, and process datasets to tokenizer.
- loading pretrained model, convert to lora model, and move model to HPU device.
- preparing data collator.
- preparing training args, an instance of `GaudiTrainingArguments`.
- preparing instance of `GaudiTrainer`.
- calling `train()` to train model.
- saving model results.


Compared to a training function for GPU, no changes are needed to port to HPU. Internally, Ray Train does these things:

- Detect HPU and set the device.
- Initialize the habana PyTorch backend.
- Initialize the habana distributed backend.

In [4]:

def train_func_per_worker(config: Dict):
    # adapt transformers to gaudi
    adapt_transformers_to_gaudi()

    # prepare datasets
    # here we use dataset "tatsu-lab/alpaca" from huggingface
    raw_datasets = datasets.DatasetDict({"train": datasets.load_dataset("tatsu-lab/alpaca", split='train[0:4096]')})
    preprocess_dataset(raw_datasets)

    # prepare tokenizer
    tokenizer = transformers.AutoTokenizer.from_pretrained(config["model"])
    tokenized_datasets = preprocess_dataset_to_tokenizer(raw_datasets, tokenizer)

    # prepare model
    model = transformers.AutoModelForCausalLM.from_pretrained(config["model"], **config["model_config"])
    peft_config = peft.LoraConfig(**config["lora_config"])
    model = peft.get_peft_model(model, peft_config)
    device = ray.train.torch.get_device()
    model.to(dtype=config["model_config"]["torch_dtype"], device=device)

    # prepare data collator
    data_collator = DataCollatorForLanguageModeling(tokenizer, pad_to_multiple_of=8, return_tensors="pt", mlm=False)

    # prepare execution mode config
    execution_mode = config["execution_mode"]
    use_lazy_mode = True if execution_mode == "lazy" else False
    torch_compile_backend = "hpu_backend" if execution_mode == "eager.compile" else None

    training_args = GaudiTrainingArguments(output_dir=config["output"], do_train=True, do_eval=False, per_device_train_batch_size=config["batch_size_per_worker"],
                                           bf16=True, learning_rate=config["lr"], save_strategy="no", torch_compile_backend=torch_compile_backend,
                                           evaluation_strategy="no", lr_scheduler_type="linear", num_train_epochs=config["epochs"],
                                           use_lazy_mode=use_lazy_mode, use_habana=True, max_grad_norm=0.3, save_only_model=True,
                                           warmup_ratio=0.03, throughput_warmup_steps=3, logging_steps=5)

    gaudi_config = GaudiConfig()
    gaudi_config.use_fused_adam = True
    gaudi_config.use_fused_clip_norm = True

    trainer = GaudiTrainer(
        model=model,
        gaudi_config=gaudi_config,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=None,
        preprocess_logits_for_metrics=None,
    )

    train_result = trainer.train()
    print(f"train_result = {train_result}")
    trainer.save_model()

    return train_result

## Main Training Function
The `train_llama` function sets up the distributed training environment using Ray and starts the training process. To enable training using HPU, we only need to make the following changes:
- Set the exectuion mode for training, supported execution mode are:

    - "lazy": Deferred execution of graphs, comprising of ops delivered from script op by op similar to Eager mode. It gives the Eager mode experience with performance on Gaudi. Unlike Eager Mode with torch.compile, graph is analyzed in each iteration leading to a higher CPU usage.
    - "eager": Op-by-op execution as defined in standard PyTorch Eager mode scripts.
    - "eager.compile": Eager mode extended with `torch.compile` - Similar to Eager mode but extended with wrapping complete or part of model (such as a function) into a graph. Parts that are not wrapped are executed eagerly.

    More detail theory can be found [here](https://docs.habana.ai/en/latest/PyTorch/Reference/PyTorch_Gaudi_Theory_of_Operations.html)
- Require an HPU for each worker in ScalingConfig
- Set backend to `hccl` in TorchConfig

In [5]:

def train_llama(num_workers, execution_mode):
    # Setting environment variables
    os.environ["RAY_EXPERIMENTAL_NOSET_HABANA_VISIBLE_MODULES"] = "true"
    if execution_mode == "lazy":
        os.environ["PT_HPU_LAZY_MODE"] = "1"
    else:
        os.environ["PT_HPU_LAZY_MODE"] = "0"

    # Preparing train configurations
    train_config = {
        "execution_mode": execution_mode,
        "model": "/root/models/llama-7b",
        "model_config": {"torch_dtype": torch.bfloat16, "trust_remote_code": False, "use_auth_token": None},
        "lora_config": {"task_type": "CAUSAL_LM", "r": 8, "lora_alpha": 32, "lora_dropout": 0.1, "target_modules": ["q_proj", "v_proj"]},
        "lr": 1e-4,
        "epochs": 2,
        "batch_size_per_worker": 8,
        "output": "/tmp/ray/",
    }

    # Configure computation resources
    # In ScalingConfig, require an HPU for each worker
    scaling_config = ScalingConfig(num_workers=num_workers, resources_per_worker={"CPU": 1, "HPU": 1})
    # Set backend to hccl in TorchConfig
    torch_config = TorchConfig(backend = "hccl")

    # start your ray cluster
    ray.init()

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        torch_config=torch_config,
        scaling_config=scaling_config,
    )

    result = trainer.fit()
    print(f"Training result: {result}")

## Start Training

Finally, we call the `train_llama` function to start the training process. You can adjust the number of workers to use, and the execution mode for HPU.

In [6]:
# execution_mode are ["lazy", "eager", "eager.compile"]
train_llama(num_workers=4, execution_mode="lazy")

(pid=678181) /usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
(pid=678181)   _torch_pytree._register_pytree_node(
(TrainTrainable pid=678181) /usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
(TrainTrainable pid=678181)   _torch_pytree._register_pytree_node(
(RayTrainWorker pid=678759) /usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
(RayTrainWorker pid=678759)   _torch_pytree._register_pytree_node(
(RayTrainWorker pid=678761) /usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:462: UserWarning: torch.utils

(RayTrainWorker pid=678759) {'loss': 2.2125, 'grad_norm': 1.6171875, 'learning_rate': 9.5e-05, 'epoch': 0.16, 'memory_allocated (GB)': 34.51, 'max_memory_allocated (GB)': 78.71, 'total_memory_available (GB)': 94.62}


 16%|█▌        | 10/62 [01:10<01:13,  1.41s/it]


(RayTrainWorker pid=678759) {'loss': 1.9621, 'grad_norm': 1.2734375, 'learning_rate': 8.666666666666667e-05, 'epoch': 0.32, 'memory_allocated (GB)': 34.51, 'max_memory_allocated (GB)': 78.72, 'total_memory_available (GB)': 94.62}


 24%|██▍       | 15/62 [01:12<00:25,  1.83it/s]


...

(RayTrainWorker pid=678760) train_result = TrainOutput(global_step=62, training_loss=1.500297857869056, metrics={'train_runtime': 93.3311, 'train_samples_per_second': 71.042, 'train_steps_per_second': 2.222, 'total_flos': 4.02963202792489e+16, 'train_loss': 1.500297857869056, 'epoch': 2.0, 'memory_allocated (GB)': 34.51, 'max_memory_allocated (GB)': 78.72, 'total_memory_available (GB)': 94.62})
(RayTrainWorker pid=678759) {'train_runtime': 93.1812, 'train_samples_per_second': 74.672, 'train_steps_per_second': 2.336, 'train_loss': 1.4982363793157762, 'epoch': 2.0, 'memory_allocated (GB)': 34.51, 'max_memory_allocated (GB)': 78.73, 'total_memory_available (GB)': 94.62}
(RayTrainWorker pid=678759) train_result = TrainOutput(global_step=62, training_loss=1.4982363793157762, metrics={'train_runtime': 93.1812, 'train_samples_per_second': 74.672, 'train_steps_per_second': 2.336, 'train_loss': 1.4982363793157762, 'epoch': 2.0, 'memory_allocated (GB)': 34.51, 'max_memory_allocated (GB)': 78.73,

100%|██████████| 62/62 [01:33<00:00,  1.50s/it]
(RayTrainWorker pid=678759) /usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/models/llama-7b - will assume that the vocabulary was not modified.
(RayTrainWorker pid=678759)   warnings.warn(


Trial TorchTrainer_c3d9a_00000 completed. Last result: 


2024-04-18 07:34:56,353	INFO tune.py:1011 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/TorchTrainer_2024-04-18_07-32-09' in 0.0056s.
2024-04-18 07:34:56,363	INFO tune.py:1043 -- Total run time: 166.88 seconds (166.82 seconds for the tuning loop).


Training result: Result(
  metrics={},
  path='/root/ray_results/TorchTrainer_2024-04-18_07-32-09/TorchTrainer_c3d9a_00000_0_2024-04-18_07-32-09',
  filesystem='local',
  checkpoint=None
)
